In [1]:
import pandas as analytics
import numpy as maths
import matplotlib.pyplot as graph
import os
from math import exp

In [2]:
os.chdir('../dependencies/')
from logistic_regression import logistic_regression

In [3]:
filename = 'data1'
validation_perc = 0.7
training_perc = 0.7
alphas = maths.arange(0,1,0.1)
number_of_iterations = 10
training_perc = 0.7

In [4]:
log_reg = logistic_regression()
log_reg.load_data(filename)
df_validation, df_test = log_reg.split_dataset(validation_perc , training_perc)
# log_reg.find_weights(alphas, number_of_iterations )
# log_reg.classify(df_test)

In [5]:
df_validation

,x0,x1,x2,y
75,1,16,22,1
171,1,37,35,0
182,1,42,5,0
192,1,50,20,0
102,1,38,27,0
...,...,...,...,...
57,1,5,22,1
112,1,37,7,0
79,1,31,42,1
51,1,11,34,1
